# 🛠️ Environment Setup & Imports

In [1]:
%pip install gradio langchain langchain-community pyttsx3 python-dotenv requests pypdf langchain-groq huggingface_hub mediapipe opencv-python pywin32

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python -m pywin32_postinstall


c:\Users\sriha\internship\storyteller\.venv_311\Scripts\python.exe: No module named pywin32_postinstall


In [4]:
# Environment Setup
import os
import sys

# Ensure current directory is in path for any relative file access needs
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

print(f"Current Working Directory: {current_dir}")


Current Working Directory: c:\Users\sriha\internship\storyteller


## Centralized Logging & CLI Formatting

This section configures structured logging, log levels, and clean terminal output formatting for the entire application.


In [5]:
import logging
import os
import sys

def setup_logger():
    """
    Configures a professional, clean logger for the application.
    Suppresses noisy libraries and handles known benign errors.
    """
    # 1. Environment & Library Suppression
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL only for TensorFlow
    
    # Check for DEBUG mode
    debug_mode = os.getenv("DEBUG", "false").lower() == "true"
    log_level = logging.DEBUG if debug_mode else logging.INFO

    # 2. Configure Main Logger
    logger = logging.getLogger("storyteller")
    logger.setLevel(log_level)
    
    # Clean Format
    formatter = logging.Formatter('[%(levelname)s] %(message)s')
    
    handler = logging.StreamHandler(sys.stdout)
    handler.setFormatter(formatter)
    
    # Avoid duplicate handlers
    if not logger.handlers:
        logger.addHandler(handler)

    # 3. Suppress Noisy Libraries
    # These libraries are very chatty; silence them unless critical
    noisy_libs = [
        "mediapipe", "tensorflow", "absl", "h5py", 
        "numexpr", "urllib3", "httpx", "httpcore"
    ]
    for lib in noisy_libs:
        logging.getLogger(lib).setLevel(logging.ERROR)

    # 4. Handle Asyncio Noise (WinError 10054)
    # This error often spams on Windows/Gradio shutdown or reload
    asyncio_logger = logging.getLogger("asyncio")
    asyncio_logger.setLevel(logging.CRITICAL) 

    return logger

def get_logger():
    return logging.getLogger("storyteller")


## Configuration & Environment Settings

This section defines global constants, model selections, API settings, and environment-level configuration used across the storytelling system.


In [16]:
import os

# LLM Models
MODEL_CREATIVE = "llama-3.3-70b-versatile"
MODEL_FAST = "llama-3.3-70b-versatile" # 8b model caused panic/instability

# Limits
MAX_HISTORY_TURNS = 10
MORAL_SCORE_MIN = -10
MORAL_SCORE_MAX = 10

# Defaults
DEFAULT_LANGUAGE = "English"

# Defaults
DEFAULT_LANGUAGE = "English"


## Model Asset Download & Setup Utility

This section downloads and prepares required external model assets (e.g., MediaPipe task files) needed for local facial emotion detection.


In [7]:
import requests
import os

url = "https://storage.googleapis.com/mediapipe-models/face_landmarker/face_landmarker/float16/1/face_landmarker.task"
output_path = "face_landmarker.task"

print(f"Downloading model from {url}...")
try:
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(output_path, "wb") as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print(f"Success: Saved to {os.path.abspath(output_path)}")
except Exception as e:
    print(f"Download failed: {e}")


Success: Saved to c:\Users\sriha\internship\storyteller\face_landmarker.task


## Cinematic Prompt & Visual Framing Engine

This section translates narrative and emotional context into cinematic descriptors such as camera angle, lighting, and atmosphere.


In [8]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
# from config import MODEL_FAST  # Cell-local import
# from logger_config import get_logger  # Cell-local import

logger = get_logger()

class CinematographyEngine:
    def __init__(self):
        # We use a specialized instance for visual instruction
        self.llm = ChatGroq(model=MODEL_FAST, temperature=0.7)

    def enhance_prompt(self, story_segment, emotion):
        """
        Generates a visually rich, cinematic description using an LLM.
        Avoids hardcoded mappings.
        """
        system_instruction = (
            "You are an expert Virtual Cinematographer and Art Director. "
            "Your task is to translate a story segment and an emotion into a precise "
            "visual description for AI Image Generation (Stable Diffusion/Flux). "
            "Focus ONLY on: Camera Angle, Lighting, Color Palette, Depth of Field, and Composition. "
            "Do NOT include the story action itself, just the visual style keywords. "
            "Keep it comma-separated and under 40 words."
        )

        prompt_template = ChatPromptTemplate.from_messages([
            ("system", system_instruction),
            ("human", "Story: {story}\nEmotion: {emotion}\n\nVisual Keywords:")
        ])

        chain = prompt_template | self.llm
        
        try:
            result = chain.invoke({"story": story_segment, "emotion": emotion})
            return result.content.strip()
        except Exception as e:
            logger.error(f"Cinematography Engine Error: {e}")
            # Fallback if LLM fails
            return f"cinematic shot, {emotion} lighting, 8k resolution"


c:\Users\sriha\internship\storyteller\.venv_311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Dynamic Cultural Context Generator

This section implements a fully generative culture engine that recalls culturally authentic concepts, values, symbols, and terminology on demand using an LLM.


In [9]:
# from config import MODEL_FAST  # Cell-local import
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
# from logger_config import get_logger  # Cell-local import

logger = get_logger()

class CultureEngine:
    def __init__(self):
        # Use Fast model for quick context retrieval/generation
        self.llm = ChatGroq(model=MODEL_FAST)

    def get_context_string(self, theme):
        """
        Dynamically generates a 'Knowledge Block' about the theme using the LLM.
        """
        if not theme:
            return ""

        logger.info(f"CultureEngine: Generatively recalling facts for '{theme}'...")
        
        system_prompt = (
            "You are an expert Cultural Anthropologist and Mythologist with encyclopedic knowledge of world cultures, "
            "folklore, and history. Your goal is to provide a concise, factual, and authentic 'Knowledge Block' "
            "that a storyteller can use to ground their narrative."
        )

        user_prompt = (
            f"Topic: {theme}\n\n"
            "Provide 9-10 key authentic cultural elements, including:\n"
            "1. Specific terminology (greetings, clothing, weapons, tools)\n"
            "2. Key festivals or rituals\n"
            "3. Mythological figures or legends\n"
            "4. Social hierarchy or values\n\n"
            "Format: A concise list or paragraph. Strictly factual and authentic. No preamble."
        )

        try:
            messages = [
                SystemMessage(content=system_prompt),
                HumanMessage(content=user_prompt)
            ]
            response = self.llm.invoke(messages)
            return response.content
        except Exception as e:
            logger.error(f"Culture Generation Failed: {e}")
            return "General cultural knowledge applies."

if __name__ == "__main__":
    ce = CultureEngine()

## Facial Emotion Detection Engine

This section detects user facial expressions in real time using MediaPipe and maps them to high-level emotional states for narrative tone adaptation.


In [10]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
import os
# from logger_config import get_logger  # Cell-local import

logger = get_logger()

class EmotionEngine:
    def __init__(self, model_path="face_landmarker.task"):
        """
        Initializes the MediaPipe FaceLandmarker with Blendshapes enabled.
        """
        try:
            if not os.path.exists(model_path):
                logger.warning(f"Model file '{model_path}' not found. Emotion detection disabled.")
                self.detector = None
                return

            base_options = python.BaseOptions(model_asset_path=model_path)
            options = vision.FaceLandmarkerOptions(
                base_options=base_options,
                output_face_blendshapes=True,
                output_facial_transformation_matrixes=False,
                num_faces=1
            )
            self.detector = vision.FaceLandmarker.create_from_options(options)
            logger.info("EmotionEngine (FaceLandmarker) initialized successfully.")
        except Exception as e:
            logger.error(f"Error initializing EmotionEngine: {e}")
            self.detector = None

    def detect_emotion(self, image):
        """
        Detects emotion from a numpy image (RGB) using Blendshapes.
        Returns: { "emotion": label, "confidence": float }
        """
        if self.detector is None or image is None:
            return {"emotion": "neutral", "confidence": 0.0}

        try:
            # Create MP Image from Numpy
            # Gradio provides RGB numpy array
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)

            # Detect
            detection_result = self.detector.detect(mp_image)

            if not detection_result.face_blendshapes:
                return {"emotion": "neutral", "confidence": 0.0}

            # Extract Blendshapes (list of categories)
            # There is 1 face, so index 0
            blendshapes = detection_result.face_blendshapes[0]
            
            # Map blendshapes to dict for easy access
            bs = {b.category_name: b.score for b in blendshapes}

            # Heuristics for Basic Emotions based on ARKit Blendshapes
            # Scores are 0.0 to 1.0
            
            scores = {
                "happy": (bs.get('mouthSmileLeft', 0) + bs.get('mouthSmileRight', 0)) / 2,
                "surprise": (bs.get('browInnerUp', 0) + bs.get('jawOpen', 0)) / 2,
                "angry": (bs.get('browDownLeft', 0) + bs.get('browDownRight', 0)) / 2,
                "sad": (bs.get('mouthFrownLeft', 0) + bs.get('mouthFrownRight', 0) + bs.get('browInnerUp', 0)) / 3,
                "fear": (bs.get('eyeWideLeft', 0) + bs.get('eyeWideRight', 0) + bs.get('mouthStretchLeft', 0)) / 3
            }

            # Find max score
            best_emotion = max(scores, key=scores.get)
            best_score = scores[best_emotion]

            # Thresholding
            if best_score < 0.3: # If signals are weak, default to neutral
                return {"emotion": "neutral", "confidence": round(1.0 - best_score, 2)}
            
            logger.info(f"Detected emotion: {best_emotion} (confidence={best_score:.2f})")
            
            return {
                "emotion": best_emotion,
                "confidence": round(best_score, 2)
            }

        except Exception as e:
            logger.debug(f"Emotion detection runtime error: {e}")
            return {"emotion": "neutral", "confidence": 0.0}


## Moral Evaluation & Karma System

This section evaluates user choices across ethical dimensions such as compassion, courage, and greed, updating the moral trajectory of the story.


In [11]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
# from config import MODEL_FAST, MORAL_SCORE_MIN, MORAL_SCORE_MAX  # Cell-local import
# from logger_config import get_logger  # Cell-local import

logger = get_logger()

class MoralScore(BaseModel):
    compassion: int = Field(description="Change in compassion score (-5 to +5)")
    courage: int = Field(description="Change in courage score (-5 to +5)")
    greed: int = Field(description="Change in greed score (-5 to +5)")
    reasoning: str = Field(description="Brief reason for the score")

class MoralEngine:
    def __init__(self):
        self.llm = ChatGroq(model=MODEL_FAST, temperature=0.5)
        self.scores = {"compassion": 0, "courage": 0, "greed": 0}
        
        self.parser = JsonOutputParser(pydantic_object=MoralScore)

    def score_choice(self, user_choice, story_context):
        """Evaluates the user's last choice."""
        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a Moral Arbiter in a story game. Analyze the user's choice and assign score changes."),
            ("human", "Story Context: {context}\nUser Choice: {choice}\n\n{format_instructions}")
        ])

        chain = prompt | self.llm | self.parser
        
        try:
            result = chain.invoke({
                "context": story_context[-500:], # Last 500 chars context
                "choice": user_choice,
                "format_instructions": self.parser.get_format_instructions()
            })
            
            # Update internal state with clamping
            for trait in ["compassion", "courage", "greed"]:
                change = result.get(trait, 0)
                new_score = self.scores[trait] + change
                # Clamp score
                self.scores[trait] = max(MORAL_SCORE_MIN, min(MORAL_SCORE_MAX, new_score))
            
            return result
        except Exception as e:
            logger.error(f"Moral Engine Error: {e}")
            return None

    def generate_reflection(self):
        """Generates a final moral summary."""
        prompt = f"""
        Based on these final scores: {self.scores},
        write a 2-sentence spiritual reflection for the player, referencing concepts like Karma or Dharma if appropriate.
        """
        response = self.llm.invoke(prompt)
        return response.content


## Dynamic Character Identity Engine

This section generates culturally grounded protagonist identities, including names, traits, and background context, based on the selected theme.


In [12]:
import random

class Character:
    def __init__(self, name, culture, age=20, traits=None, voice_id="21m00Tcm4TlvDq8ikWAM", face_seed=None):
        self.id = f"char_{random.randint(1000, 9999)}"
        self.name = name
        self.culture = culture
        self.age = age
        self.traits = traits if traits else []
        self.voice_id = voice_id
        # Persistent seed for image generation consistency
        self.face_seed = face_seed if face_seed else random.randint(10000, 99999)
        
    def add_trait(self, trait):
        if trait not in self.traits:
            self.traits.append(trait)

    def to_dict(self):
        return {
            "id": self.id,
            "name": self.name,
            "culture": self.culture,
            "age": self.age,
            "traits": self.traits,
            "voice_id": self.voice_id,
            "face_seed": self.face_seed
        }

    @staticmethod
    def from_dict(data):
        return Character(
            name=data["name"],
            culture=data["culture"],
            age=data["age"],
            traits=data["traits"],
            voice_id=data["voice_id"],
            face_seed=data["face_seed"]
        )

# from config import MODEL_FAST  # Cell-local import
from langchain_groq import ChatGroq
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
# from logger_config import get_logger  # Cell-local import

logger = get_logger()

class CharacterIdentity(BaseModel):
    name: str = Field(description="A culturally appropriate name for the protagonist")
    culture_label: str = Field(description="A formally normalized culture label (e.g., 'Indian Epic - Ramayana')")

class CharacterEngine:
    def __init__(self):
        self.llm = ChatGroq(model=MODEL_FAST)
        self.parser = JsonOutputParser(pydantic_object=CharacterIdentity)

    def _generate_identity_llm(self, theme_input):
        """Generates dynamic character identity using LLM."""
        try:
            prompt = (
                f"Analyze the theme '{theme_input}'.\n"
                "Generate a culturally authentic protagonist name and a formal culture label.\n"
                "Example: 'samurai' -> Name: 'Kenji', Culture: 'Japanese History - Samurai Era'\n"
                f"{self.parser.get_format_instructions()}"
            )
            response = self.llm.invoke(prompt)
            data = self.parser.parse(response.content)
            return data["name"], data["culture_label"]
        except Exception as e:
            logger.error(f"Identity Generation Failed: {e}")
            return "Protagonist", theme_input.title()

    def initialize_character(self, name, culture_input):
        # If name is generic or missing, use LLM to generate identity
        if not name or "Protagonist" in name:
            gen_name, gen_culture = self._generate_identity_llm(culture_input)
            final_name = gen_name
            final_culture = gen_culture
        else:
            final_name = name
            final_culture = culture_input.title()
            
        return Character(final_name, final_culture)

    def get_visual_description(self, character):
        """Returns a stable visual description for the image prompt."""
        traits_str = ", ".join(character.traits)
        return (
            f"character {character.name}, {character.culture} ethnicity, "
            f"age {character.age}, wearing traditional attire, {traits_str}, "
            f"consistent face, high detail"
        )

    def update_traits_from_scores(self, character, scores):
        """Updates character traits based on moral alignment."""
        new_traits = []
        
        # Compassion
        c_score = scores.get('compassion', 0)
        if c_score >= 5:
            new_traits.append("Kind")
        elif c_score <= -5:
            new_traits.append("Ruthless")
            
        # Courage
        co_score = scores.get('courage', 0)
        if co_score >= 5:
            new_traits.append("Brave")
        elif co_score <= -5:
            new_traits.append("Cowardly")
            
        # Greed
        g_score = scores.get('greed', 0)
        if g_score >= 5:
            new_traits.append("Ambitious")
        elif g_score <= -5:
            new_traits.append("Generous")
            
        for trait in new_traits:
            character.add_trait(trait)
        
        return character


## Media Generation & Narration Engine

This section handles optional audio narration and image generation, with graceful fallbacks when media capabilities are unavailable.


In [13]:
import os
import time
import traceback
import requests
import base64
import pyttsx3
# from cinematography_engine import CinematographyEngine  # Cell-local import
# from logger_config import get_logger  # Cell-local import

logger = get_logger()

class MediaEngine:
    def __init__(self):
        # Hugging Face token check
        self.hf_token = os.getenv("HUGGINGFACE_API_TOKEN")
        if not self.hf_token:
            logger.warning("HUGGINGFACE_API_TOKEN not found. Image generation disabled.")

        # pyttsx3 is initialized per-call to avoid threading issues on Windows/Gradio

        # Initialize Cinematography Engine
        try:
            self.cine_engine = CinematographyEngine()
        except Exception as e:
            logger.error(f"Failed to init CinematographyEngine: {e}")
            self.cine_engine = None

    # ---------------- IMAGE GENERATION ----------------
    def generate_scene(self, story_text, emotion="neutral", character_desc="", visual_keywords_bypass=None):
        if not self.hf_token:
            return None, "image"

        try:
            # 1. Get Cinematography Keywords
            if visual_keywords_bypass:
                keywords = visual_keywords_bypass
            elif self.cine_engine:
                keywords = self.cine_engine.enhance_prompt(story_text[:500], emotion)
            else:
                keywords = f"cinematic, {emotion} atmosphere"

            # 2. Construct Prompt with Character Consistency
            prompt = f"""
            masterpiece, ultra-detailed cinematic illustration, storybook fantasy art,
            authentic cultural aesthetics, rich textures, 8k resolution,
            {keywords},
            
            SCENE:
            {story_text[:400]}
            
            CHARACTER FOCUS:
            {character_desc}
            
            STYLE:
            digital painting, concept art, unreal engine quality, artstation trending
            
            NEGATIVE:
            blurry, low resolution, distorted face, extra limbs, bad anatomy, watermark, text
            """

            API_URL = (
                "https://router.huggingface.co/hf-inference/models/"
                "black-forest-labs/FLUX.1-schnell"
            )
            headers = {
                "Authorization": f"Bearer {self.hf_token}",
                "Content-Type": "application/json"
            }

            response = requests.post(
                API_URL,
                headers=headers,
                json={"inputs": prompt},
                timeout=60
            )

            if response.status_code != 200:
                raise Exception(f"HF Error {response.status_code}: {response.text}")

            filename = f"scene_{int(time.time())}.png"
            output_path = os.path.abspath(filename)

            with open(output_path, "wb") as f:
                f.write(response.content)

            logger.info(f"Image saved → {output_path}")
            return output_path, "image"

        except Exception as e:
            logger.error(f"Image generation failed: {e}")
            logger.debug(traceback.format_exc())

            with open("debug_error.log", "w") as f:
                f.write(str(e))

            return None, "image"

    # ---------------- AUDIO GENERATION ----------------
    def generate_audio(self, text, voice_id=None):
        try:
            # Re-initialize pyttsx3 per call for thread safety in Gradio
            engine = pyttsx3.init()
            # Optional: Set properties
            engine.setProperty('rate', 150)
            engine.setProperty('volume', 1.0)
            
            output_path = f"audio_{int(time.time())}.mp3"
            abs_output_path = os.path.abspath(output_path)
            
            # Saving to file
            engine.save_to_file(text, abs_output_path)
            engine.runAndWait()
            
            # Explicit cleanup if possible (pyttsx3 doesn't have a close(), but letting it go out of scope helps)
            del engine

            return abs_output_path

        except Exception as e:
            logger.error(f"Audio generation failed: {e}")
            return None

    # ---------------- VIDEO GENERATION ----------------



## Core Narrative Generation Engine

This section contains the main storytelling logic, combining cultural context, character identity, emotion, and moral state to generate adaptive narratives.


In [14]:
# from config import MODEL_CREATIVE, MAX_HISTORY_TURNS  # Cell-local import
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage
# from culture_engine import CultureEngine  # Cell-local import
# from logger_config import get_logger  # Cell-local import

logger = get_logger()

class StoryOutput(BaseModel):
    story_text: str = Field(description="The narrative content (100-150 words) with choices at the end")
    emotion: str = Field(description="One word emotion: joy, sadness, anger, fear, peace, mystery")
    visual_keywords: str = Field(description="Comma-separated visual keywords(Camera Angle, Lighting, Color Palette)")

class StoryTeller:
    def __init__(self):
        self.llm = ChatGroq(model=MODEL_CREATIVE)
        self.history = []
        self.culture_engine = CultureEngine()
        self.language_instruction = "Narrate in English."
        self.parser = JsonOutputParser(pydantic_object=StoryOutput)

    def set_language(self, language="English"):
        if language and language.lower() != "english":
            self.language_instruction = (
                f"Narrate primarily in English, BUT you MUST adhere to the following code-switching rules:\n"
                f"1. Use {language} for ALL opening greetings and significant cultural terms.\n"
                f"2. Quotes and dialogue MUST be in {language} (provide English translation in parentheses if long).\n"
                f"3. Ensure the tone reflects the linguistic nuance of {language}.\n"
                f"Example: 'Namaste! (Hello!) The wind howled...' "
            )
        else:
            self.language_instruction = "Narrate in English."

    def _trim_history(self):
        """Trims history to keep only the most recent interactions."""
        try:
            # Keep system prompt (index 0) + last N turns
            if len(self.history) > MAX_HISTORY_TURNS:
                # Safely slice the last (N-1) elements
                keep_count = MAX_HISTORY_TURNS - 1
                recent_history = self.history[-keep_count:]
                self.history = [self.history[0]] + recent_history
        except Exception as e:
            logger.warning(f"History Trimming Error: {e}")
            # Fallback: Just keep explicit last 5
            if len(self.history) > 5:
                self.history = [self.history[0]] + self.history[-5:]

    def start_story(self, theme, language="English"):
        """Initializes the story based on a theme and cultural context."""
        self.set_language(language)
        
        # 1. Retrieve Cultural Context (RAG)
        logger.info(f"Retrieving cultural context for: {theme}")
        context_str = self.culture_engine.get_context_string(theme)
        
        if context_str:
            grounding_instruction = (
                "You have access to the following trusted cultural knowledge:\n"
                f"{context_str}\n\n"
                "CRITICAL INSTRUCTION: You MUST ground your story in this provided context. "
                "Use specific symbols, names, and festivals mentioned. "
                "Do NOT hallunicate details if they contradict this context."
            )
        else:
            grounding_instruction = "No specific cultural documents found. Rely on general knowledge but remain respectful and authentic."

        # 2. Build System Prompt
        system_prompt = (
            "You are a 'Smart Cultural Storyteller'. Your goal is to preserve and retell cultural narratives "
            "in an engaging, interactive 'choose-your-own-adventure' style.\n\n"
            f"{grounding_instruction}\n\n"
            f"Language Rule: {self.language_instruction}\n\n"
            "Format:\n"
            "- Keep responses concise (100-150 words).\n"
            "- End with exactly 2 or 3 distinct choices.\n"
            "- CRITICAL: Format choices as numbered list: 1. [Choice A] 2. [Choice B] etc.\n"
            "- If information is unknown, acknowledge it subtly or steer towards known elements.\n"
            "OUTPUT JSON ONLY: Return a valid JSON object with keys: 'story_text', 'emotion', 'visual_keywords'."
        )

        self.history = [SystemMessage(content=system_prompt)]
        
        prompt = f"Start a story about {theme}. Set the scene and offer numbered choices.\n{self.parser.get_format_instructions()}"
        self.history.append(HumanMessage(content=prompt))
        
        try:
            response = self.llm.invoke(self.history)
            self.history.append(response)
            parsed_response = self.parser.parse(response.content)
            return parsed_response
        except Exception as e:
            logger.error(f"Story Start Error: {e}")
            # Fallback
            return {
                "story_text": f"The story begins with {theme}. (Error generating full story)",
                "emotion": "mystery",
                "visual_keywords": "foggy, ancient, mysterious"
            }

    def continue_story(self, user_choice):
        """Continues the story based on user's choice."""
        self._trim_history()
        self.history.append(HumanMessage(content=user_choice))
        
        try:
            response = self.llm.invoke(self.history)
            self.history.append(response)
            parsed_response = self.parser.parse(response.content)
            return parsed_response
        except Exception as e:
            logger.error(f"Story Continue Error: {e}")
            return {
                "story_text": "The story continues... (Error generating segment)",
                "emotion": "neutral",
                "visual_keywords": "standard scene"
            }


## Application Orchestration & Interactive UI

This section serves as the main entry point, orchestrating all engines and launching the interactive Gradio-based storytelling interface.


In [ ]:
import gradio as gr
import time
import os
from dotenv import load_dotenv
# from logger_config import setup_logger, get_logger  # Cell-local import

# 1. Setup Professional Logging (Must be first)
setup_logger()
logger = get_logger()

# 2. Startup Banner
logger.info("---------------------------------------------------------------")
logger.info("   📖 SMART CULTURAL STORYTELLER 2.0")
logger.info("   Research-Grade Engine | Python 3.11 | Groq | MediaPipe")
logger.info("---------------------------------------------------------------")
logger.info("System initializing...")

# from story_engine import StoryTeller  # Cell-local import
# from media_engine import MediaEngine  # Cell-local import
# from character_engine import CharacterEngine, Character  # Cell-local import
# from moral_engine import MoralEngine  # Cell-local import
# from emotion_engine import EmotionEngine  # Cell-local import

# Load environment variables
load_dotenv()

# Initialize Engines
if not os.getenv("GOOGLE_API_KEY"):
    logger.error("GOOGLE_API_KEY not found.")
    exit(1)

story_teller = StoryTeller()
media_engine = MediaEngine()
character_engine = CharacterEngine()
try:
    emotion_engine = EmotionEngine()
    if emotion_engine.detector is None:
         logger.warning("Emotion Engine initialized but detector is None.")
except Exception as e:
    logger.warning(f"Emotion Engine failed to load ({e}). Face detection disabled.")
    emotion_engine = None

logger.info("Engine validation complete. Launching UI...")

def start_story_handler(theme, language, history_state):
    try:
        if not theme:
            yield "Please enter a theme.", None, None, history_state, "", ""
            return
        
        # Initialize Character & Moral Engine
        char_name = f"Protagonist_{theme.split()[0]}"
        character = character_engine.initialize_character(char_name, theme)
        moral = MoralEngine()

        # Start Story (Returns JSON dict)
        story_data = story_teller.start_story(theme, language)
        story_text = story_data.get("story_text", "")
        emotion = story_data.get("emotion", "neutral")
        visual_keywords = story_data.get("visual_keywords")
        
        # Immediate yield: Story Text
        yield story_text, None, None, {
            "character": character.to_dict(), 
            "moral_scores": moral.scores
        }, "Compassion: 0 | Courage: 0 | Greed: 0", ""

        # Generate Audio
        audio = media_engine.generate_audio(story_text)
        yield story_text, audio, None, {
            "character": character.to_dict(), 
            "moral_scores": moral.scores
        }, "Compassion: 0 | Courage: 0 | Greed: 0", ""

        # Generate Image (Optimized)
        char_desc = character_engine.get_visual_description(character)
        media_path, media_type = media_engine.generate_scene(story_text, emotion, char_desc, visual_keywords_bypass=visual_keywords)
        
        image_update = gr.update(value=media_path, visible=True) if media_type == "image" else gr.update(visible=False)

        yield (
            story_text, 
            audio, 
            image_update,
            {
                "character": character.to_dict(), 
                "moral_scores": moral.scores
            },
            f"Compassion: 0 | Courage: 0 | Greed: 0",
            "" # Status msg
        )

    except Exception as e:
        logger.error(f"Error in start_story_handler: {e}")
        yield f"Error: {e}", None, None, None, "", ""


def process_emotion_stream(image, last_time):
    """
    Background process to detect emotion from stream.
    Throttled to run every 1.0s to avoid log spam/CPU load.
    Returns: (new_emotion_label, new_timestamp)
    """
    current_time = time.time()
    
    # Throttle: Only process if 1.0s passed
    if image is None or (current_time - last_time < 1.0):
        return gr.skip(), gr.skip()
        
    if emotion_engine:
        try:
            # This calling detect_emotion will trigger the print log if confidence > 0.3
            result = emotion_engine.detect_emotion(image)
            label = result["emotion"] if result else "neutral"
            return label, current_time
        except Exception:
            return "neutral", current_time
            
    return "neutral", current_time


def continue_story_handler(user_choice, user_emotion_label, state):
    try:
        if not user_choice:
            yield "Please make a choice.", None, None, state, "", ""
            return

        # Rehydrate State
        if not state:
            yield "Session expired. Start over.", None, None, None, "", ""
            return
        
        character = Character.from_dict(state["character"])
        moral = MoralEngine()
        moral.scores = state["moral_scores"]

        # 1. Score the Choice
        moral_result = moral.score_choice(user_choice, story_teller.history[-1].content)
        
        # 2. Update Character Traits
        character = character_engine.update_traits_from_scores(character, moral.scores)

        # 3. Continue Story (Returns JSON)
        # Use the passed-in emotion label (from State)
        
        # Inject into context
        context_choice = f"{user_choice} (User Facial Emotion: {user_emotion_label})"
        
        story_data = story_teller.continue_story(context_choice)
        story_text = story_data.get("story_text", "")
        # Blend Emotions: Story > Facial
        story_emotion = story_data.get("emotion", "neutral")
        final_emotion = story_emotion if story_emotion != "neutral" else user_emotion_label
        visual_keywords = story_data.get("visual_keywords")
        
        moral_display = f"Compassion: {moral.scores['compassion']} | Courage: {moral.scores['courage']} | Greed: {moral.scores['greed']}"
        
        status_msg = f"✨ Karma Updated! (Compassion: {moral_result.get('compassion')}, Courage: {moral_result.get('courage')}, Greed: {moral_result.get('greed')}) | Face: {user_emotion_label}"

        # Yield Text immediately
        yield story_text, None, None, state, moral_display, status_msg
        
        # 4. Generate Audio
        audio = media_engine.generate_audio(story_text)
        yield story_text, audio, None, state, moral_display, status_msg

        # 5. Generate Media
        char_desc = character_engine.get_visual_description(character)
        media_path, media_type = media_engine.generate_scene(story_text, final_emotion, char_desc, visual_keywords_bypass=visual_keywords)
        image_update = gr.update(value=media_path, visible=True) if media_type == "image" else gr.update(visible=False)

        # Update State
        state["moral_scores"] = moral.scores
        state["character"] = character.to_dict()

        # Check if story ended
        if "THE END" in story_text.upper():
            reflection = moral.generate_reflection()
            story_text += f"\n\n✨ **Moral Reflection**: {reflection}"

        yield story_text, audio, image_update, state, moral_display, status_msg

    except Exception as e:
        logger.error(f"Error in continue_story_handler: {e}")
        yield f"Error: {e}", None, None, state, "", ""


# Gradio Interface
with gr.Blocks(title="Smart Cultural Storyteller") as demo:
    state = gr.State({})
    # State to hold background emotion detection
    emotion_state = gr.State("neutral")
    # State to hold last processing timestamp for throttling
    timer_state = gr.State(0.0)

    gr.Markdown("# 📖 Smart Cultural Storyteller")
    gr.Markdown("Research-Grade Interactive Storytelling with RAG, Cinematography, and Moral Agents.")
    
    with gr.Row():
        with gr.Column(scale=1):
            # Input Section
            theme_input = gr.Textbox(label="Culture / Theme", placeholder="Indian Folklore, Samurai Legends...")
            lang_input = gr.Dropdown(["English", "Hindi", "Telugu", "Japanese", "Spanish"], label="Language", value="English")
            # Enable Streaming
            webcam_input = gr.Image(label="Your Emotion (Optional)", sources=["webcam"], type="numpy", visible=True, streaming=True)
            # Oral History Removed
            
            start_btn = gr.Button("Start New Journey", variant="primary")
            
            gr.Markdown("---")
            
            # Game Controls
            choice_input = gr.Textbox(label="Your Choice / Action")
            continue_btn = gr.Button("Make Choice")

            # Stats Display
            gr.Markdown("### ⚖️ Moral Alignment")
            moral_info = gr.Textbox(interactive=False, label="Karma Score")
            status_info = gr.Textbox(interactive=False, label="Recent Updates")
            
        with gr.Column(scale=2):
            story_display = gr.Markdown(label="Story")
            
            with gr.Row():
                image_display = gr.Image(label="Illustration", type="filepath", visible=False)
            
            audio_display = gr.Audio(label="Narration", type="filepath", autoplay=True)

    # Event Handlers
    
    # Streaming Event
    webcam_input.stream(
        fn=process_emotion_stream,
        inputs=[webcam_input, timer_state],
        outputs=[emotion_state, timer_state],
        show_progress=False
    )

    start_btn.click(
        fn=start_story_handler,
        inputs=[theme_input, lang_input, state],
        outputs=[story_display, audio_display, image_display, state, moral_info, status_info]
    )
    
    continue_btn.click(
        fn=continue_story_handler,
        inputs=[choice_input, emotion_state, state],
        outputs=[story_display, audio_display, image_display, state, moral_info, status_info]
    )

if __name__ == "__main__":
    logger.info("Starting Web Server at http://127.0.0.1:7860...")
    demo.launch(theme=gr.themes.Soft(), quiet=True) # quiet to suppress some Gradio logs


[INFO] ---------------------------------------------------------------
[INFO]    📖 SMART CULTURAL STORYTELLER 2.0
[INFO]    Research-Grade Engine | Python 3.11 | Groq | MediaPipe
[INFO] ---------------------------------------------------------------
[INFO] System initializing...
[INFO] EmotionEngine (FaceLandmarker) initialized successfully.
[INFO] Engine validation complete. Launching UI...
[INFO] Starting Web Server at http://127.0.0.1:7860...


[INFO] Retrieving cultural context for: japan
[INFO] CultureEngine: Generatively recalling facts for 'japan'...
[ERROR] Image generation failed: HF Error 402: {"error":"You have reached the free monthly usage limit for hf-inference. Subscribe to PRO to get 20x more included usage, or add pre-paid credits to your account."}
